In [1]:
!wget http://hca.gilead.org.il/vt/li_merma.txt

--2024-12-28 18:33:01--  http://hca.gilead.org.il/vt/li_merma.txt
Resolving hca.gilead.org.il (hca.gilead.org.il)... 152.19.134.41
Connecting to hca.gilead.org.il (hca.gilead.org.il)|152.19.134.41|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50061 (49K) [text/plain]
Saving to: ‘li_merma.txt’

li_merma.txt        100%[===================>]  48.89K  --.-KB/s    in 0.1s    

2024-12-28 18:33:02 (343 KB/s) - ‘li_merma.txt’ saved [50061/50061]



In [2]:
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec

In [3]:
with open("li_merma.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [4]:
sentences = [simple_preprocess(sentence) for sentence in text.splitlines() if sentence]

In [5]:
model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

In [7]:
model.wv.most_similar("mermaid", topn=5)

[('the', 0.9987457990646362),
 ('and', 0.998675525188446),
 ('she', 0.9986720085144043),
 ('we', 0.9986239671707153),
 ('her', 0.9986210465431213)]

In [8]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
sentences = [
    [word for word in simple_preprocess(sentence) if word not in stop_words]
    for sentence in text.splitlines() if sentence
]

In [10]:
model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

In [11]:
model.wv.most_similar("mermaid", topn=5)

[('others', 0.4231564402580261),
 ('sighed', 0.28460729122161865),
 ('much', 0.28006765246391296),
 ('said', 0.27466458082199097),
 ('noise', 0.2731291651725769)]

In [12]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

In [13]:
dictionary = Dictionary(sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]

In [15]:
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [16]:
topics = lda_model.print_topics(num_words=5)
for idx, topic in topics:
    print(f"Topic {idx}: {topic}")

Topic 0: 0.022*"could" + 0.019*"mermaid" + 0.019*"little" + 0.016*"never" + 0.012*"one"
Topic 1: 0.021*"sea" + 0.013*"little" + 0.012*"would" + 0.010*"soul" + 0.010*"immortal"
Topic 2: 0.026*"water" + 0.014*"beautiful" + 0.013*"great" + 0.010*"fish" + 0.009*"flowers"
Topic 3: 0.022*"prince" + 0.018*"little" + 0.016*"came" + 0.010*"like" + 0.009*"mermaid"
Topic 4: 0.020*"sea" + 0.013*"said" + 0.010*"prince" + 0.010*"water" + 0.010*"quite"


In [17]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 31.8 MB/s eta 0:00:00


In [18]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

lda_display = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)

pyLDAvis.display(lda_display)

In [19]:
pyLDAvis.save_html(lda_display, 'lda_visualization.html')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
